In [9]:
from fairness.matching import compute
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from fairness.pre_processing import categorical_to_numeric_converter

In [2]:
dataset = pd.read_csv('./dataset/adult.data', sep=',')

In [10]:
dataset = categorical_to_numeric_converter(dataset)

In [11]:
dataset.rename(columns={" <=50K": "50K"}, inplace=True)

In [12]:
output_column = "50K"

In [13]:
X = dataset.loc[:,:output_column]
y = dataset.loc[:, output_column:]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
standard_scaler_train = StandardScaler()
X_train = standard_scaler_train.fit_transform(X_train)

standard_scaler_test = StandardScaler()
X_test = standard_scaler_test.fit_transform(X_test)

In [16]:
model = GaussianNB()
model.fit(X_train, y_train)

/home/antonio/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [18]:
y_pred = model.predict(X)
dataset['y_pred'] = y_pred

/home/antonio/.local/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


In [19]:
print(confusion_matrix(y, y_pred))
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

[[24705    14]
 [ 3388  4453]]
0.8955159705159705
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     24719
           1       1.00      0.57      0.72      7841

    accuracy                           0.90     32560
   macro avg       0.94      0.78      0.83     32560
weighted avg       0.91      0.90      0.88     32560

